In [1]:
from summa import summarizer
from summa import keywords

In [84]:
text="“Borrowing” means (a) Revolving Loans of the same Type and currency, made, converted or continued on the same date and, in the case of Eurocurrency Loans, CDOR Rate Loans or EURIBOR Loans, as to which a single Interest Period is in effect or (b) a Swingline Loan."
text1="whether such Borrowing is to be an ABR Borrowing, a Eurocurrency Borrowing, a EURIBOR Borrowing, or a CDOR Rate Borrowing, as applicable;"
text2=" Borrowing request should be addressed to appropriate Applicable Agent. 2 Borrowings may be denominated in US Dollars or an Alternative Currency. Borrowings by Canadian Borrowers must be denominated in US Dollars or Canadian Dollars. 3 The principal amount of any Borrowing must be an integral multiple of the Borrowing Multiple and not less than the Borrowing Minimum. 4 The date of any Borrowing must be a Business Day and (a) in the case of a Eurocurrency Borrowing, CDOR Rate Borrowing or EURIBOR Borrowing, at least the third Business Day after the date of this Borrowing Request, and (b) in the case of an ABR Borrowing, the date of this Borrowing Request."

In [85]:
text=text.replace("\n"," ")

In [82]:
summarizer.summarize(text,ratio=0.9)

''

In [176]:
kw=keywords.keywords(text,ratio=0.9)
for i in kw.split("\n"):
    print(i)

borrowing means
loans
loan
revolving
euribor
swingline
eurocurrency
cdor


In [86]:
text

'“Borrowing” means (a) Revolving Loans of the same Type and currency, made, converted or continued on the same date and, in the case of Eurocurrency Loans, CDOR Rate Loans or EURIBOR Loans, as to which a single Interest Period is in effect or (b) a Swingline Loan.'

In [87]:
import re
from nltk.corpus import stopwords

In [166]:
stopwords_list=stopwords.words('english')
def sort_dict(dict_with_val):
    return sorted(dict_with_val.items(), key=lambda x: x[1], reverse=True)
def count_vectorizer(text_arr,weighted=True):
    freq={}
    max_freq=0
    for sent in text_arr:
        if len(sent.split())<=1:
            continue
        for words in sent.split():
            if words in stopwords_list:
                continue
            else:
                if words in freq:

                    freq[words]+=1
                else:
                    freq[words]=1
                if freq[words]>max_freq:
                    max_freq=freq[words]
    if weighted:
        freq={i:freq[i]/max_freq for i in freq}
    return freq
def get_score(freq,text_arr):
    score_dict={}
    for sent in text_arr:
        score=0
        words=0
        for word in sent.split():
            if word in freq:
                score+=freq[word]
                words+=1
        if words>0:
            score_dict[sent]=score/words
    return score_dict
def text_processing(text,separator=False):
    replace_char=['”',"`"]
    
    for i in replace_char:
        text=text.replace(i,"")
    text=re.sub("[^A-Za-z ]","<br>",text)
    if separator:
        return text
    text_arr=text.split("<br>")
    
    text_arr=[i.strip().lower() for i in text_arr if len(i.strip())>3]
    return text_arr
def freq_summazier(text,top=5):
    text_arr=text_processing(text)
    freq=count_vectorizer(text_arr)
    score_dict=get_score(freq,text_arr)
    score_dict=sort_dict(score_dict)
    #print(score_dict[:top])
    return [i[0] for i in score_dict[:top]]

In [167]:
freq_summazier(text)

['cdor rate loans or euribor loans',
 'in the case of eurocurrency loans',
 'revolving loans of the same type and currency',
 'borrowing means',
 'converted or continued on the same date and']

In [108]:
from nltk.cluster.util import cosine_distance
import pandas as pd
import numpy as np
import networkx as nx

In [168]:
def _vectorize(sent,max_len):
    all_words=sent.split()
    vector=[0] * max_len
    for w in all_words:
        if w in stopwords_list:
            continue
        vector[all_words.index(w)] += 1
    return vector
def _similarity_matrix(vector_space):
    matrix=[]
    for i in vector_space:
        vector_similarity=[]
        for j in vector_space:
            #print(i,j,cosine_distance(i,j))
            vector_similarity.append(1-cosine_distance(i,j))
        matrix.append(vector_similarity)
    return matrix
def page_rank(similarity_matrix):
    nx_graph = nx.from_numpy_array(np.array(similarity_matrix))
    scores = nx.pagerank(nx_graph)
    return scores
def textRank(sent):
    text_arr=text_processing(sent)
    vector_space=[]
    max_len=max([len(sent.split()) for sent in text_arr ])
    for sent in text_arr:
        vector_space.append(_vectorize(sent,max_len=max_len))
    similarity_matrix=_similarity_matrix(vector_space)
    scores=page_rank(similarity_matrix)
    sent_score={i:scores[idx] for idx,i in enumerate(text_arr)}
    score_dict=sort_dict(sent_score)

    return score_dict

In [169]:
textRank(text2)

[('the date of any borrowing must be a business day and', 0.10705547632228526),
 ('borrowings may be denominated in us dollars or an alternative currency',
  0.10426551634560821),
 ('the principal amount of any borrowing must be an integral multiple of the borrowing multiple and not less than the borrowing minimum',
  0.10344470400293228),
 ('the date of this borrowing request', 0.10295042251343602),
 ('in the case of a eurocurrency borrowing', 0.10166993922026026),
 ('in the case of an abr borrowing', 0.10166993922026024),
 ('borrowings by canadian borrowers must be denominated in us dollars or canadian dollars',
  0.09841237387609286),
 ('at least the third business day after the date of this borrowing request',
  0.09409936571855615),
 ('cdor rate borrowing or euribor borrowing', 0.09364945581050019),
 ('borrowing request should be addressed to appropriate applicable agent',
  0.09278280697006862)]

In [111]:
scores

{0: 0.13247615705577656,
 1: 0.13136116783000956,
 2: 0.11225903134770922,
 3: 0.13708671374825437,
 4: 0.11847398791237398,
 5: 0.15539974830061665,
 6: 0.09175459614920446,
 7: 0.12118859765605493}

In [184]:
def degree(text_arr):
    deg={}
    for sent in text_arr:
        word_array=[word for word in sent.split() if word not in stopwords_list]
        if len(word_array)<=1:
            continue
        for words in word_array:
            if words in deg:
                deg[words]+=len(word_array)
            else:
                deg[words]=len(word_array)
    return deg     
def rake_score(deg,freq):
    score={}
    for word,degree in deg.items():
        score[word]=degree/freq[word]
    return score
def rake_sent_score(text_arr,score_dict,norm=True):
    text_score={}
    for sent in text_arr:
        sent_score=0
        words=sent.split()
        for word in words:
            if word in score_dict:
                sent_score+=score_dict[word]
        if norm:
            text_score[sent]=sent_score/len(words)
        else:
            text_score[sent]=sent_score
    return text_score
def RAKE_SENT(text):
    text_arr=text_processing(text)
    freq=count_vectorizer(text_arr,weighted=False)
    deg=degree(text_arr)
    score=rake_score(deg,freq)
    score_dict=rake_sent_score(text_arr,score)
    score_dict=sort_dict(score_dict)
    
    return score_dict
def RAKE(text):
    text_arr=text_processing(text)
    freq=count_vectorizer(text_arr,weighted=False)
    deg=degree(text_arr)
    score=rake_score(deg,freq)
    score=sort_dict(score)
    return score

In [185]:
RAKE_SENT(text2)

[('borrowings by canadian borrowers must be denominated in us dollars or canadian dollars',
  5.846153846153846),
 ('borrowings may be denominated in us dollars or an alternative currency',
  4.848484848484849),
 ('the principal amount of any borrowing must be an integral multiple of the borrowing multiple and not less than the borrowing minimum',
  4.746556473829201),
 ('cdor rate borrowing or euribor borrowing', 4.621212121212121),
 ('borrowing request should be addressed to appropriate applicable agent',
  3.9663299663299663),
 ('at least the third business day after the date of this borrowing request',
  3.2843822843822843),
 ('the date of any borrowing must be a business day and', 2.881542699724518),
 ('the date of this borrowing request', 2.7828282828282824),
 ('in the case of a eurocurrency borrowing', 1.7662337662337662),
 ('in the case of an abr borrowing', 1.7662337662337662)]

In [177]:
RAKE(text1)

[('whether', 4.0),
 ('abr', 4.0),
 ('borrowing', 3.0),
 ('cdor', 3.0),
 ('rate', 3.0),
 ('eurocurrency', 2.0),
 ('euribor', 2.0)]